<a href="https://colab.research.google.com/github/Bgail-ranu/Codveda_Technologies/blob/main/Level1_Task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Load Data**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [3]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/Stock_Prices_Data_Set.csv')
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,symbol,date,open,high,low,close,volume
0,AAL,2014-01-02,25.0700,25.8200,25.0600,25.3600,8998943
1,AAPL,2014-01-02,79.3828,79.5756,78.8601,79.0185,58791957
2,AAP,2014-01-02,110.3600,111.8800,109.2900,109.7400,542711
3,ABBV,2014-01-02,52.1200,52.3300,51.5200,51.9800,4569061
4,ABC,2014-01-02,70.1100,70.2300,69.4800,69.8900,1148391


# **Inspect Data**

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497472 entries, 0 to 497471
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   symbol  497472 non-null  object 
 1   date    497472 non-null  object 
 2   open    497461 non-null  float64
 3   high    497464 non-null  float64
 4   low     497464 non-null  float64
 5   close   497472 non-null  float64
 6   volume  497472 non-null  int64  
dtypes: float64(4), int64(1), object(2)
memory usage: 26.6+ MB


In [5]:
df.shape

(497472, 7)

In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
open,497461.0,8.635227e+01,1.014712e+02,1.62,4.169000e+01,6.497000e+01,98.41,2.044000e+03
high,497464.0,8.713256e+01,1.023121e+02,1.69,4.209000e+01,6.556000e+01,99.23,2.067990e+03
low,497464.0,8.555247e+01,1.005710e+02,1.50,4.128000e+01,6.435370e+01,97.58,2.035110e+03
close,497472.0,8.636908e+01,1.014724e+02,1.59,4.170375e+01,6.498000e+01,98.42,2.049000e+03
volume,497472.0,4.253611e+06,8.232139e+06,0.00,1.080166e+06,2.084896e+06,4271928.00,6.182376e+08


In [7]:
df.isnull().sum()

,0
symbol,0
date,0
open,11
high,8
low,8
close,0
volume,0


In [8]:
df.columns

Index(['symbol', 'date', 'open', 'high', 'low', 'close', 'volume'], dtype='object')

In [9]:
df['symbol'].value_counts()

,count
symbol,
YUM,1007
XYL,1007
XRX,1007
XRAY,1007
XOM,1007
...,...
DXC,189
BHGE,126
BHF,117


# **Handling missing values**

In [10]:
df = df.dropna(subset=['open'])
df = df.dropna(subset=['high'])
df = df.dropna(subset=['low'])
df.shape, df.isnull().sum()

((497461, 7),
 symbol    0
 date      0
 open      0
 high      0
 low       0
 close     0
 volume    0
 dtype: int64)

In [11]:
df.duplicated().sum()

np.int64(0)

In [12]:
df['date'].value_counts().sort_index()

,count
date,
2014-01-02,483
2014-01-03,483
2014-01-06,483
2014-01-07,483
2014-01-08,483
...,...
2017-12-22,505
2017-12-26,505
2017-12-27,505


# **Feature creation**

In [15]:
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['month_name'] = df['date'].dt.month_name()
df['day_of_week'] = df['date'].dt.day_name()
df['quarter'] = df['date'].dt.quarter

# Check the results
print(df[['date', 'year', 'month', 'month_name', 'day_of_week']].head())

        date  year  month month_name day_of_week
0 2014-01-02  2014      1    January    Thursday
1 2014-01-02  2014      1    January    Thursday
2 2014-01-02  2014      1    January    Thursday
3 2014-01-02  2014      1    January    Thursday
4 2014-01-02  2014      1    January    Thursday


In [16]:
df.head()

,symbol,date,open,high,low,close,volume,year,month,month_name,day_of_week,quarter
0,AAL,2014-01-02,25.0700,25.8200,25.0600,25.3600,8998943,2014,1,January,Thursday,1
1,AAPL,2014-01-02,79.3828,79.5756,78.8601,79.0185,58791957,2014,1,January,Thursday,1
2,AAP,2014-01-02,110.3600,111.8800,109.2900,109.7400,542711,2014,1,January,Thursday,1
3,ABBV,2014-01-02,52.1200,52.3300,51.5200,51.9800,4569061,2014,1,January,Thursday,1
4,ABC,2014-01-02,70.1100,70.2300,69.4800,69.8900,1148391,2014,1,January,Thursday,1


In [17]:
df.to_csv('Stock_DataSet_cleaned.csv', index=False)

In [18]:
gdrive_path = '/content/drive/MyDrive/Stock_DataSet_cleaned.csv'

df.to_csv(gdrive_path, index=False)

print(f"A new permanent file has been saved to: {gdrive_path}")

A new permanent file has been saved to: /content/drive/MyDrive/Stock_DataSet_cleaned.csv


In [ ]:
df = df.drop(columns=['date', 'month', 'day_of_week', 'quarter'])
df

# **Preprocessing**

## **Split Data**

In [25]:
y = df['volume']
x = df.drop('volume', axis=1)
y

,volume
0,8998943
1,58791957
2,542711
3,4569061
4,1148391
...,...
497467,1046677
497468,1347613
497469,1023624
497470,1261916


In [26]:
x

,symbol,open,high,low,close,year,month_name
0,AAL,25.0700,25.8200,25.0600,25.3600,2014,January
1,AAPL,79.3828,79.5756,78.8601,79.0185,2014,January
2,AAP,110.3600,111.8800,109.2900,109.7400,2014,January
3,ABBV,52.1200,52.3300,51.5200,51.9800,2014,January
4,ABC,70.1100,70.2300,69.4800,69.8900,2014,January
...,...,...,...,...,...,...,...
497467,XYL,68.5300,68.8000,67.9200,68.2000,2017,December
497468,YUM,82.6400,82.7100,81.5900,81.6100,2017,December
497469,ZBH,121.7500,121.9500,120.6200,120.6700,2017,December
497470,ZION,51.2800,51.5500,50.8100,50.8300,2017,December


## **Categorizing and encoding**

In [29]:
categorical_features = x.select_dtypes(include=["object"]).columns
numerical_features = x.select_dtypes(include=["int64", "float64"]).columns

categorical_features, numerical_features

(Index(['symbol', 'month_name'], dtype='object'),
 Index(['open', 'high', 'low', 'close'], dtype='object'))

In [30]:
scaler = StandardScaler()
encoder = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers = [
        ("num", scaler, numerical_features),
        ("cat", encoder, categorical_features)
    ]
)

## **Train-Test Split**

In [31]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((397968, 7), (99493, 7), (397968,), (99493,))